In [ ]:
import pandas as pd
import tkinter as tk
from tkinter import ttk
import os
import tkinter.messagebox
import customtkinter
import unicodedata
import requests
import tempfile
import chardet
from CTkTable import *
from tkinter import filedialog
from PIL import Image, ImageTk
from tkintertable.Tables import TableCanvas
from pandasgui import show
from pandastable import Table, TableModel
from pandastable.data import TableModel
# from pandastable.plugins import DataMergePlugin
from simpledbf import Dbf5
from io import BytesIO
import matplotlib.pyplot as plt
import time


# Definindo a aparência padrão como "light"
customtkinter.set_appearance_mode("light")
customtkinter.set_default_color_theme("green")

class DataFrameTable(Table):
    def __init__(self, parent=None, dataframe=None, showtoolbar=True, showstatusbar=True, **kwargs):
        model = TableModel(dataframe)
        Table.__init__(self, parent, model=model, showtoolbar=showtoolbar, showstatusbar=showstatusbar, **kwargs)

class AppSaudados(customtkinter.CTk):
    def __init__(self):
        super().__init__()

        # Definindo a aparência padrão como "light"
        customtkinter.set_appearance_mode("light")
        customtkinter.set_default_color_theme("green")

        # Definindo a escala inicial como 80%
        customtkinter.set_widget_scaling(0.8)

        self.title("SAUDADOS")
        self.geometry(f"{1100}x{580}")
        # self.resizable(width=False, height=False)

        self.grid_columnconfigure(1, weight=1)
        self.grid_columnconfigure((1, 2), weight=0)
        self.grid_rowconfigure((0, 1, 2), weight=1)

        self.sidebar_frame = customtkinter.CTkFrame(self, width=140, corner_radius=0)
        self.sidebar_frame.grid(row=0, column=0, rowspan=10, sticky="nsew")
        self.sidebar_frame.grid_rowconfigure(4, weight=1)

        #titulo
        self.logo_label = customtkinter.CTkLabel(self.sidebar_frame, text="SAUDADOS – CIÊNCIA DE DADOS E SAÚDE", font=customtkinter.CTkFont(size=12, weight="bold"))
        self.logo_label.grid(row=0, column=0, padx=20, pady=(20, 10))

        #entrada
        self.Entrada_label = customtkinter.CTkLabel(self.sidebar_frame, text="Olá, Bem Vindo!", font=customtkinter.CTkFont(size=10, weight="bold"))
        self.Entrada_label.grid(row=1, column=0, padx=20, pady=(10))

        texto = "O objetivo desse sistema é apoiar as ações de pesquisa\n\n" +"utilizando dados de bases da saúde, bem como dados\n\n" +"demográficos entre outros.\n\n" + "Instruções:\n\n"
        
        # Usando Label para exibir o texto
        self.labelTexto = customtkinter.CTkLabel(self.sidebar_frame, text=texto, justify="left", font=customtkinter.CTkFont(size=10))
        self.labelTexto.grid(row=2, column=0, padx=(20, 0), pady=(20, 0), sticky="nsew")

        # Cria um arquivo temporário para armazenar temporariamente o DataFrame
        self.temp_file = tempfile.NamedTemporaryFile(delete=False)
        self.dataframe1 = tempfile.NamedTemporaryFile(delete=False)
        self.dataframe2 = tempfile.NamedTemporaryFile(delete=False)

        self.dataframes_carregados = []
        self.colunas_selecionadas_primeiro_arquivo = []
        self.colunas_selecionadas_segundo_arquivo = []

        # Variável de controle de cancelamento
        self._cancel_load = False

        #mudar tema
        self.appearance_mode_label = customtkinter.CTkLabel(self.sidebar_frame, text="Aparência:", anchor="w")
        self.appearance_mode_label.grid(row=5, column=0, padx=20, pady=(10, 0))
        self.appearance_mode_optionemenu = customtkinter.CTkOptionMenu(self.sidebar_frame, values=["Light", "Dark", "System"],
                                                                    command=self.change_appearance_mode_event)
        self.appearance_mode_optionemenu.grid(row=6, column=0, padx=20, pady=(10, 10))
        self.scaling_label = customtkinter.CTkLabel(self.sidebar_frame, text="Escala:", anchor="w")
        self.scaling_label.grid(row=7, column=0, padx=20, pady=(10, 0))
        self.scaling_optionemenu = customtkinter.CTkOptionMenu(self.sidebar_frame, values=["80%", "90%", "100%", "110%", "120%"],
                                                            command=self.change_scaling_event)
        self.scaling_optionemenu.grid(row=8, column=0, padx=20, pady=(10, 20))

        #fundo  
        self.imagem_label = customtkinter.CTkFrame(self, width=200, corner_radius=0)
        self.imagem_label.grid(row=0, column=1, rowspan=5, padx=(10, 0), pady=(0, 0), sticky="nsew")
        self.imagem_label.grid_rowconfigure(4, weight=1)

        #base de dados
        self.label_base_dados = customtkinter.CTkLabel(self.imagem_label, text="Selecione a Base de Dados:", anchor="w")
        self.label_base_dados.grid(row=0, column=0, padx=(40, 0), pady=(10, 0))

        # Obter todos os arquivos de um diretório específico
        diretorio = "D:/Area de Trabalho/Repositorys/PIBITI-IFBA-CNPq-2023-2024/ProjetoSaudados/BaseDadosTeste-Saudados"  
        arquivos = [arq for arq in os.listdir(diretorio) if os.path.isfile(os.path.join(diretorio, arq))]

        self.menu_base_dados = customtkinter.CTkOptionMenu(self.imagem_label, values=[" ", *arquivos], command=self.on_base_dados_selected)
        self.menu_base_dados.grid(row=1, column=0, padx=(40, 0), pady=(10, 10))

        # EXTENSAO
        self.label_extensao = customtkinter.CTkLabel(self.imagem_label, text="Selecione a Extensão:", anchor="w")
        self.label_extensao.grid(row=2, column=0, padx=(40, 0), pady=(10, 0))
        self.menu_extensao = customtkinter.CTkOptionMenu(self.imagem_label, values=[" ", ".csv", ".xlsx", ".json", ".dbf", ".txt"], command=self.on_extensao_selected)
        self.menu_extensao.grid(row=3, column=0, padx=(40, 0), pady=(10, 10))

        #ISO OU UTF-8
        self.label_codificacao = customtkinter.CTkLabel(self.imagem_label, text="Selecione o Tipo de Upload:", anchor="w")
        self.label_codificacao.grid(row=4, column=0, padx=(40, 0), pady=(10, 0))
        self.menu_codificacao = customtkinter.CTkOptionMenu(self.imagem_label, values=[" ", "iso", "utf-8"], command=self.on_codificacao_selected)
        self.menu_codificacao.grid(row=5, column=0, padx=(40, 0), pady=(10, 10))

        #Exclusão de acentos ou caracteres especiais;
        self.checkbox_caracteres = customtkinter.CTkCheckBox(self.imagem_label, text="Exclusão de acentos\n" + "ou caracteres especiais")
        self.checkbox_caracteres.grid(row=6, column=0, pady=(90,0), padx=(20,0), sticky="n")

        #Botao ler arquivo
        self.button_LerArquivo = customtkinter.CTkButton(self.imagem_label, text="Ler Arquivo", command=self.Ler_Arquivo_DataFrame)
        self.button_LerArquivo.grid(row=8, column=0, padx=(40, 0), pady=(10, 20))

        # Debug
        self.label_debug = customtkinter.CTkFrame(self, width=50, corner_radius=0)
        self.label_debug.grid(row=6, column=1, columnspan=2, sticky="nsew")
        self.label_debug.grid_rowconfigure(4, weight=1)

        # Debug entry
        self.entry = customtkinter.CTkLabel(self.label_debug, text="Selecione um arquivo")
        self.entry.grid(row=6, column=1)

        self.progress_bar = tkinter.ttk.Progressbar(self.label_debug, orient='horizontal', length=500, mode='determinate')
        self.progress_bar.grid(row=7, column=1, padx=20, pady=(10, 10))

        # Scrollable Frame para as colunas
        self.scrollable_frame = customtkinter.CTkScrollableFrame(self, label_text="Selecione as Colunas:")
        self.scrollable_frame.grid(row=0, column=2, padx=(20, 0), pady=(20, 0), sticky="nsew")
        self.scrollable_frame.grid_columnconfigure(0, weight=1)
        self.scrollable_frame_switches = []

        #join
        self.checkbox_JOIN = customtkinter.CTkCheckBox(self, text="Realizar Join")
        self.checkbox_JOIN.grid(row=1, column=2, pady=(20, 0), padx=0, sticky="n")
        self.checkbox_JOIN.grid_remove()  # Inicia o checkbox_JOIN como oculto

        # Botão para carregar as colunas selecionadas
        self.button_carregar_colunas = customtkinter.CTkButton(self, text="Carregar DataFrame", command=self.carregar_colunas_selecionadas)
        self.button_carregar_colunas.grid(row=1, column=2, padx=20, pady=50)

        # Adicionando variáveis para armazenar os valores dos switches
        self.switch_variables = []
    
    def dbf2DF(self, file_path):
        dbf = Dbf5(file_path)
        return dbf.to_dataframe()

     # Função para alterar o modo de aparência
    def change_appearance_mode_event(self, new_appearance_mode):
        customtkinter.set_appearance_mode(new_appearance_mode)

    def change_scaling_event(self, new_scaling):
        new_scaling_float = int(new_scaling.replace("%", "")) / 100
        customtkinter.set_widget_scaling(new_scaling_float)

    @staticmethod
    def remove_special_characters(text):
        text = ''.join(c for c in unicodedata.normalize('NFKD', text) if not unicodedata.combining(c))
        return text.upper()
    
    def determinar_delimitador(self, arquivo):
        with open(arquivo, 'r') as file:
            line = file.readline()
            delimiters = [',', ';', '\t']
            for delimiter in delimiters:
                if delimiter in line:
                    return delimiter
            return ';'
        
    def on_base_dados_selected(self, value):
        self.selected_file = value  # Armazenar o arquivo selecionado ao escolher na opção
        self.entry.configure(text=f"Selecionado o arquivo {self.selected_file}")

    def on_extensao_selected(self, value):
        self.selected_extension = value
        self.entry.configure(text=f"Selecionada a extensão: {self.selected_extension}")

    def on_codificacao_selected(self, value):
        self.selected_codificacao = value
        self.entry.configure(text=f"Selecionada a codificação: {self.selected_codificacao}")

    def on_Barra_de_Carregamento(self):
        self.progress_bar['value'] = 0
        self.entry.configure(text="Carregando...")
        
        # Simulação de carregamento com barra de progresso
        for i in range(100):
            time.sleep(0.05)  # Simulando uma operação demorada
            self.progress_bar['value'] = i + 1
            self.update_idletasks()  # Atualiza a interface para exibir a barra de progresso
        
        self.entry.configure(text="Carregamento concluído.")
        self.progress_bar['value'] = 0  # Reinicia a barra de progresso

    def mostrar_colunas_disponiveis(self, df):
        columns = df.columns.tolist()  # Obtém os nomes das colunas como uma lista
        # Após a leitura bem-sucedida do arquivo CSV, cria os interruptores para as colunas
        for i, col in enumerate(columns):
            switch_var = tkinter.IntVar(value=0)  # Variável para armazenar o valor do interruptor
            switch = customtkinter.CTkSwitch(master=self.scrollable_frame, text=f"{col}", variable=switch_var)
            switch.grid(row=i, column=0, padx=10, pady=(0, 20))
            self.scrollable_frame_switches.append(switch)
            self.switch_variables.append(switch_var)

    def mostrar_colunas_exibidas(self, df):
        """Mostra colunas que estão sendo exibidas na tela."""
        colunas_exibidas = []
        for switch_var, switch in zip(self.switch_variables, self.scrollable_frame_switches):
            if switch_var.get() == 1:  # Verifica se o interruptor está ativado
                colunas_exibidas.append(switch.cget("text"))  # Obter o nome da coluna

        return colunas_exibidas  # Retorna a lista de colunas que estão sendo exibidas

    def Ler_Arquivo_DataFrame(self):
        if self.selected_extension == ".csv":
            file_path = filedialog.askopenfilename(initialdir="/", title="Selecione um arquivo",
                                                    filetypes=(("Arquivos CSV", ".csv"), ("Todos os arquivos", ".*")))
            if file_path:  # Verifica se um arquivo foi selecionado
                self.on_Barra_de_Carregamento()

                try:
                    with open(file_path, 'r', encoding='utf-8') as file:
                        if self.selected_codificacao == "iso":
                            df_temp = pd.read_csv(file, delimiter=self.determinar_delimitador(file_path), low_memory=False)

                            # # Converte todas as colunas para números se possível
                            # df_temp = df_temp.apply(pd.to_numeric, errors='ignore')

                            self.mostrar_colunas_disponiveis(df_temp)

                            # Armazena temporariamente o DataFrame no arquivo temporário
                            df_temp.to_csv(self.temp_file.name, index=False)

                            self.entry.configure(text="Arquivo carregado com sucesso! Visualize as colunas disponíveis.")
                        elif self.selected_codificacao == "utf-8":
                            df_temp = pd.read_csv(file, delimiter=self.determinar_delimitador(file_path), encoding='utf-8', low_memory=False)

                            # # Converte todas as colunas para números se possível
                            # df_temp = df_temp.apply(pd.to_numeric, errors='coerce')

                            self.mostrar_colunas_disponiveis(df_temp)

                            # Armazena temporariamente o DataFrame no arquivo temporário
                            df_temp.to_csv(self.temp_file.name, index=False)

                            self.entry.configure(text="Arquivo carregado com sucesso! Visualize as colunas disponíveis.")
                        else:
                            self.entry.configure(text="Nenhum tipo de codificação selecionado")

                except UnicodeDecodeError as e:
                    self.entry.configure(text=f"Erro ao ler o arquivo: {str(e)}. Certifique-se de escolher a codificação correta.")
                except Exception as e:
                    self.entry.configure(text=f"Erro ao carregar arquivo: {str(e)}")
            else:
                self.entry.configure(text="Nenhum arquivo selecionado.")
        elif self.selected_extension == ".xls":
            file_path = filedialog.askopenfilename(initialdir="/", title="Selecione um arquivo Excel",
                                                    filetypes=(("Arquivos Excel", ".xls .xlsx"), ("Todos os arquivos", ".*")))
            if file_path:
                self.on_Barra_de_Carregamento()

                if self.selected_codificacao == "iso":
                    # Lê o arquivo Excel usando o pandas
                    df_temp = pd.read_excel(file_path, low_memory=False)

                    # Converte todas as colunas para números se possível
                    df_temp = df_temp.apply(pd.to_numeric, errors='ignore')

                    self.mostrar_colunas_disponiveis(df_temp)

                    df_temp.to_csv(self.temp_file.name, index=False)

                    self.entry.configure(text="Arquivo Excel carregado com sucesso! Visualize as colunas disponíveis.")

                elif self.selected_codificacao == "utf-8":
                    df_temp = pd.read_excel(file_path, low_memory=False, encoding='utf-8')
                
                    # Converte todas as colunas para números se possível
                    df_temp = df_temp.apply(pd.to_numeric, errors='ignore')

                    self.mostrar_colunas_disponiveis(df_temp)

                    df_temp.to_csv(self.temp_file.name, index=False)

                    self.entry.configure(text="Arquivo Excel carregado com sucesso! Visualize as colunas disponíveis.")
                else:
                    self.entry.configure(text="Nenhun tipo de codificação selecionado")
            else:
                self.entry.configure(text="Nenhum arquivo Excel selecionado.")

        elif self.selected_extension == ".json":
            file_path = filedialog.askopenfilename(initialdir="/", title="Selecione um arquivo JSON",
                                                    filetypes=(("Arquivos JSON", ".json"), ("Todos os arquivos", ".*")))
            if file_path:
                self.on_Barra_de_Carregamento()

                if self.selected_codificacao == "iso":
                    # Lê o arquivo JSON usando o pandas
                    df_temp = pd.read_json(file_path, orient='records', lines=True, encoding='latin-1')

                    # Converte todas as colunas para números se possível
                    df_temp = df_temp.apply(pd.to_numeric, errors='ignore')

                    self.mostrar_colunas_disponiveis(df_temp)

                    df_temp.to_csv(self.temp_file.name, index=False)

                    self.entry.configure(text="Arquivo JSON carregado com sucesso! Visualize as colunas disponíveis.")

                elif self.selected_codificacao == "utf-8":
                    # Lê o arquivo JSON usando o pandas
                    df_temp = pd.read_json(file_path, orient='records', lines=True, encoding='utf-8')
                
                    # Converte todas as colunas para números se possível
                    df_temp = df_temp.apply(pd.to_numeric, errors='ignore')

                    self.mostrar_colunas_disponiveis(df_temp)

                    df_temp.to_csv(self.temp_file.name, index=False)

                    self.entry.configure(text="Arquivo JSON carregado com sucesso! Visualize as colunas disponíveis.")
                else:
                    self.entry.configure(text="Nenhun tipo de codificação selecionado")
            else:
                self.entry.configure(text="Nenhum arquivo JSON selecionado.")

        elif self.selected_extension == ".dbf":
            file_path = filedialog.askopenfilename(initialdir="/", title="Selecione um arquivo DBF",
                                                    filetypes=(("Arquivos DBF", ".dbf"), ("Todos os arquivos", ".*")))
            if file_path:
                self.on_Barra_de_Carregamento()

                # Usa a função dbf2DF para ler o arquivo DBF
                df_temp = self.dbf2DF(file_path)

                if self.selected_codificacao == "iso":
                    df_temp.to_csv(self.temp_file.name, index=False)

                    # Converte todas as colunas para números se possível
                    df_temp = df_temp.apply(pd.to_numeric, errors='ignore')

                    self.mostrar_colunas_disponiveis(df_temp)

                    self.entry.configure(text="Arquivo DBF carregado com sucesso! Visualize as colunas disponíveis.")
                elif self.selected_codificacao == "utf-8":
                    # Exemplo: Salva o DataFrame em um arquivo CSV com encoding 'utf-8'
                    df_temp.to_csv(self.temp_file.name, index=False, encoding='latin-1')

                    # Converte todas as colunas para números se possível
                    df_temp = df_temp.apply(pd.to_numeric, errors='ignore')

                    self.mostrar_colunas_disponiveis(df_temp)

                    self.entry.configure(text="Arquivo DBF carregado com sucesso! Visualize as colunas disponíveis.")
                else:
                    self.entry.configure(text="Nenhun tipo de codificação selecionado")
            else:
                self.entry.configure(text="Nenhum arquivo DBF selecionado.")

        elif self.selected_extension == ".txt":
            file_path = filedialog.askopenfilename(initialdir="/", title="Selecione um arquivo de texto",
                                                    filetypes=(("Arquivos de Texto", ".txt"), ("Todos os arquivos", ".*")))
            if file_path:
                self.on_Barra_de_Carregamento()

                if self.selected_codificacao == "iso":
                    # Lê o conteúdo do arquivo de texto usando a função open() com encoding 'latin-1'
                    with open(file_path, 'r', encoding='latin-1') as file:
                        # Lê todas as linhas do arquivo e remove os caracteres de quebra de linha
                        lines = [line.strip() for line in file.readlines()]

                        # Separa cada linha em colunas usando um delimitador (por exemplo, tabulação)
                        # Você precisa substituir '\t' pelo delimitador correto usado no seu arquivo de texto
                        df_temp = pd.DataFrame([line.split('\t') for line in lines])

                        # Define o cabeçalho do DataFrame como a primeira linha do arquivo de texto
                        df_temp.columns = df_temp.iloc[0]
                        df_temp = df_temp[1:]  # Remove a primeira linha (cabeçalho) do DataFrame

                    self.mostrar_colunas_disponiveis(df_temp)
                    df_temp.to_csv(self.temp_file.name, index=False)

                    self.entry.configure(text="Arquivo de texto carregado com sucesso! Visualize as colunas disponíveis.")
                elif self.selected_codificacao == "utf-8":
                    # Lê o conteúdo do arquivo de texto usando a função open() com encoding 'utf-8'
                    with open(file_path, 'r', encoding='utf-8') as file:
                        lines = [line.strip() for line in file.readlines()]
                        df_temp = pd.DataFrame([line.split('\t') for line in lines])
                        df_temp.columns = df_temp.iloc[0]
                        df_temp = df_temp[1:]

                    self.mostrar_colunas_disponiveis(df_temp)
                    df_temp.to_csv(self.temp_file.name, index=False)

                    self.entry.configure(text="Arquivo de texto carregado com sucesso! Visualize as colunas disponíveis.")
                else:
                    self.entry.configure(text="Nenhum tipo de codificação selecionado")
            else:
                self.entry.configure(text="Nenhum arquivo de texto selecionado.")

        else:
            self.entry.configure(text="Selecione um arquivo CSV, Excel, JSON, DBF ou de texto antes de ler!")

    def carregar_colunas_selecionadas(self):
        if not os.path.exists(self.temp_file.name):
            self.entry.configure(text="Nenhum arquivo temporário encontrado. Carregue um arquivo antes de selecionar colunas.")
            return

        colunas_selecionadas = [self.scrollable_frame_switches[i].cget("text") for i, var in enumerate(self.switch_variables) if var.get()]

        if not colunas_selecionadas:
            self.entry.configure(text="Selecione pelo menos uma coluna para carregar.")
            return

        # Carrega o DataFrame com as colunas selecionadas
        df_selecionado = pd.read_csv(self.temp_file.name, usecols=colunas_selecionadas, low_memory=False)
        self.dataframes_carregados.append(df_selecionado)

        # Verifica se já foi perguntado sobre carregar mais bases de dados
        if not self.checkbox_JOIN.winfo_ismapped():  # Verifica se o checkbox_JOIN está oculto
            resposta = tkinter.messagebox.askquestion("SAUDADOS", "Deseja carregar mais alguma base de dados?")
            if resposta == 'yes':
                # Mostra a opção de JOIN e salva o DataFrame atual em dataframe1
                self.checkbox_JOIN.grid()
                self.dataframe1 = tempfile.NamedTemporaryFile(delete=False)
                df_selecionado.to_csv(self.dataframe1.name, index=False)

                # Reinicia o processo de leitura para o segundo arquivo
                self.scrollable_frame_switches = []  # Limpa os switches anteriores
                self.switch_variables = []  # Limpa as variáveis dos switches
                self.temp_file = tempfile.NamedTemporaryFile(delete=False)  # Cria um novo arquivo temporário para o novo DataFrame
                self.entry.configure(text="Carregue um novo arquivo para realizar o JOIN.")
                print("Arquivo 0:", self.temp_file.name)
                print("Arquivo 1:", self.dataframe1.name)
                print("Arquivo 2:", self.dataframe2.name)
            else:
                print("Arquivo 0:", self.temp_file.name)
                print("Arquivo 1:", self.dataframe1.name)
                print("Arquivo 2:", self.dataframe2.name)
                # Se a resposta for não, mostra o DataFrame atual
                self.mostrar_dataframe_selecionado(df_selecionado, "DataFrame Principal")
                self.entry.configure(text="Operação finalizada.")
        else:
            # Verifica se dataframe1 já foi salvo
            if not self.dataframe1.name:
                df_selecionado = pd.read_csv(self.temp_file.name, usecols=colunas_selecionadas, low_memory=False)
                df_selecionado.to_csv(self.dataframe1.name, index=False)
                self.temp_file = tempfile.NamedTemporaryFile(delete=False)  # Reinicia temp_file para o segundo arquivo
                print("DataFrame 1 carregado")
            else:
                df_selecionado = pd.read_csv(self.temp_file.name, usecols=colunas_selecionadas, low_memory=False)
                df_selecionado.to_csv(self.dataframe2.name, index=False)
                print("DataFrame 2 carregado")
                
                # Exibe os dois DataFrames, passando o nome dos arquivos
                self.mostrar_ambos_dataframes(self.selected_file, "Nome do Segundo Arquivo")
                
            print("Arquivo 0:", self.temp_file.name)
            print("Arquivo 1:", self.dataframe1.name)
            print("Arquivo 2:", self.dataframe2.name)
            
            self.entry.configure(text="Operação concluída. DataFrames carregados.")
    
    def mostrar_ambos_dataframes(self, nome_arquivo1, nome_arquivo2):
        # Ler os DataFrames
        df1 = pd.read_csv(self.dataframe1.name)
        df2 = pd.read_csv(self.dataframe2.name)

        janela_dataframe = customtkinter.CTkToplevel(self)
        janela_dataframe.title("Comparando DataFrames")
        janela_dataframe.geometry(f"{1100}x{580}")

        # Frame superior para informações do JOIN, caso o JOIN esteja habilitado
        if self.checkbox_JOIN.get():
            frame_superior = customtkinter.CTkFrame(janela_dataframe, height=100)
            frame_superior.grid(row=0, column=0, columnspan=2, padx=10, pady=(10, 0), sticky="nsew")

            # Rótulo para instrução
            label_join_info = customtkinter.CTkLabel(frame_superior, text="Selecione o tipo de JOIN e as colunas para realizar o JOIN:",
                                                    font=customtkinter.CTkFont(size=14, weight="bold"))
            label_join_info.grid(row=0, column=0, columnspan=2, padx=10, pady=5)

            # Menu para selecionar o tipo de JOIN
            self.menu_tipo_join = customtkinter.CTkOptionMenu(frame_superior, values=["Inner Join", "Left Join", "Right Join", "Outer Join"])
            self.menu_tipo_join.grid(row=1, column=0, padx=10, pady=10)

            # Menu para selecionar a coluna de JOIN do DataFrame 1
            colunas_df1 = df1.columns.tolist()
            self.menu_coluna_df1 = customtkinter.CTkOptionMenu(frame_superior, values=colunas_df1)
            self.menu_coluna_df1.grid(row=1, column=1, padx=10, pady=10)

            # Menu para selecionar a coluna de JOIN do DataFrame 2
            colunas_df2 = df2.columns.tolist()
            self.menu_coluna_df2 = customtkinter.CTkOptionMenu(frame_superior, values=colunas_df2)
            self.menu_coluna_df2.grid(row=1, column=2, padx=10, pady=10)

            # Botão para realizar o JOIN
            self.botao_realizar_join = customtkinter.CTkButton(frame_superior, text="Realizar JOIN", command=self.executar_join)
            self.botao_realizar_join.grid(row=1, column=3, padx=10, pady=10)

        # Exibindo DataFrame 1
        frame1 = customtkinter.CTkFrame(janela_dataframe)
        frame1.grid(row=1, column=0, padx=10, pady=10, sticky="nsew")

        label_df1 = customtkinter.CTkLabel(frame1, text=f"DataFrame 1 - {nome_arquivo1}", font=customtkinter.CTkFont(size=14, weight="bold"))
        label_df1.grid(row=0, column=0, pady=(10, 5))  

        # Usando o Table diretamente para exibir o DataFrame 1
        pt1 = Table(frame1, dataframe=df1)
        pt1.show()

        # Exibindo DataFrame 2
        frame2 = customtkinter.CTkFrame(janela_dataframe)
        frame2.grid(row=1, column=1, padx=10, pady=10, sticky="nsew")

        label_df2 = customtkinter.CTkLabel(frame2, text=f"DataFrame 2 - {nome_arquivo2}", font=customtkinter.CTkFont(size=14, weight="bold"))
        label_df2.grid(row=0, column=0, pady=(10, 5))  

        # Usando o Table diretamente para exibir o DataFrame 2
        pt2 = Table(frame2, dataframe=df2)
        pt2.show()

        # Configurações de layout para ambos os DataFrames
        janela_dataframe.grid_columnconfigure(0, weight=1)
        janela_dataframe.grid_columnconfigure(1, weight=1)
        janela_dataframe.grid_rowconfigure(1, weight=1)

        # Crie um Frame para agrupar o rótulo e a barra de progresso
        frame_inferior = customtkinter.CTkFrame(janela_dataframe)
        frame_inferior.grid(row=2, column=0, columnspan=2, padx=20, pady=10, sticky="ew")

        # Adiciona o rótulo no Frame inferior
        self.entry = customtkinter.CTkLabel(frame_inferior, text="Selecione o Tipo de Join")
        self.entry.grid(row=0, column=0, columnspan=2, padx=20, pady=10, sticky="ew")

        # Adiciona a barra de progresso no Frame inferior
        progress_bar = ttk.Progressbar(frame_inferior, orient='horizontal', length=500, mode='determinate')
        progress_bar.grid(row=1, column=0, columnspan=2, padx=20, pady=10, sticky="ew")

    # Função para atualizar o feedback de tipo de JOIN
    def update_join_feedback(self, tipo_join):
        self.entry.configure(text=f"Tipo de JOIN selecionado: {tipo_join}")

    # Função para atualizar o feedback de seleção de colunas
    def update_column_feedback(self, coluna, dataframe):
        self.entry.configure(text=f"Coluna {coluna} selecionada para o {dataframe}")

    # Atualizando a função de executar_join para fornecer feedback de progresso
    def executar_join(self):
        # Feedback de início do JOIN
        self.entry.configure(text="Iniciando o JOIN...")
        self.progress_bar['value'] = 0
        self.update_idletasks()

        # Simulando progresso do JOIN (exemplo)
        for i in range(50):
            time.sleep(0.05)  # Simulando processamento
            self.progress_bar['value'] += 2
            self.update_idletasks()

        # Obter as opções selecionadas
        tipo_join = self.menu_tipo_join.get()
        coluna_df1 = self.menu_coluna_df1.get()
        coluna_df2 = self.menu_coluna_df2.get()

        # Ler os DataFrames
        df1 = pd.read_csv(self.dataframe1.name)
        df2 = pd.read_csv(self.dataframe2.name)

        # Verificar o tipo de JOIN e realizá-lo
        if tipo_join == "Inner Join":
            resultado = df1.merge(df2, left_on=coluna_df1, right_on=coluna_df2, how="inner")
        elif tipo_join == "Left Join":
            resultado = df1.merge(df2, left_on=coluna_df1, right_on=coluna_df2, how="left")
        elif tipo_join == "Right Join":
            resultado = df1.merge(df2, left_on=coluna_df1, right_on=coluna_df2, how="right")
        elif tipo_join == "Outer Join":
            resultado = df1.merge(df2, left_on=coluna_df1, right_on=coluna_df2, how="outer")
        else:
            tkinter.messagebox.showerror("Erro", "Selecione um tipo de JOIN válido")
            return

        # Feedback de conclusão do JOIN
        self.entry.configure(text="JOIN concluído!")
        self.progress_bar['value'] = 100
        self.update_idletasks()

        # Chama a função para mostrar o DataFrame resultante do JOIN em uma nova janela
        self.mostrar_dataframe_selecionado(resultado, "Resultado do JOIN")



    def mostrar_dataframe_selecionado(self, dataframe, nome_arquivo):
        janela_dataframe = customtkinter.CTkToplevel(self)
        janela_dataframe.title(f"DataFrame - {nome_arquivo}")
        janela_dataframe.geometry(f"{1100}x{580}")

        # Divide a janela em duas colunas
        janela_dataframe.grid_columnconfigure(0, weight=1)  # Coluna 0
        janela_dataframe.grid_columnconfigure(1, weight=3)  # Coluna 1 (larger)

        # Divide a janela em duas linhas
        janela_dataframe.grid_rowconfigure(0, weight=1)  # Linha 0
        janela_dataframe.grid_rowconfigure(1, weight=3)  # Linha 1 (larger)

        # Parte menor: Adiciona os botões para gerar relatórios e gráficos
        self.parte_menor = customtkinter.CTkFrame(janela_dataframe, width=200, corner_radius=0)
        self.parte_menor.grid(row=0, column=0, rowspan=2, sticky="nsew", padx=(0, 10))  # Adiciona padx para criar o espaçamento
        self.parte_menor.grid_rowconfigure(4, weight=1)

        # Botão para gerar relatório
        self.botao_relatorio = customtkinter.CTkButton(self.parte_menor, text="Gerar Relatório", command=self.gerar_relatorios)
        self.botao_relatorio.grid(row=0, column=0, padx=10, pady=(10, 20))

        # Botão para gerar gráficos
        self.botao_graficos = customtkinter.CTkButton(self.parte_menor, text="Gerar Gráficos", command=self.abrir_janela_graficos)
        self.botao_graficos.grid(row=1, column=0, padx=10, pady=(10, 20))

        # Parte maior: Exibe o DataFrame
        self.parte_maior = customtkinter.CTkFrame(janela_dataframe, width=200, corner_radius=0)
        self.parte_maior.grid(row=0, column=1, rowspan=2, sticky="nsew", padx=(10, 0))  # Adiciona padx para criar o espaçamento
        self.parte_maior.grid_rowconfigure(4, weight=1)

        # Adiciona a tabela como um widget
        self.tabela_dataframe = DataFrameTable(self.parte_maior, dataframe=dataframe, showtoolbar=True, showstatusbar=True, editable=False)
        self.tabela_dataframe.grid(row=0, column=0, columnspan=3, padx=20, pady=20)

        try:
            # Mostrar a tabela dentro da janela
            self.tabela_dataframe.show()
        except Exception as e:
            print(f"Erro ao mostrar a tabela: {e}")

        # Atualiza a interface
        janela_dataframe.update_idletasks()

    def abrir_janela_graficos(self):
        # Usa o último DataFrame carregado da lista `dataframes_carregados`
        if not self.dataframes_carregados:
            tkinter.messagebox.showerror("Erro", "Nenhum DataFrame foi carregado.")
            return

        dataframe = self.dataframes_carregados[-1]  # Usa o último DataFrame carregado

        # Janela para selecionar tipo de gráfico e colunas
        janela_graficos = customtkinter.CTkToplevel(self)
        janela_graficos.title("Selecionar Gráfico")
        janela_graficos.geometry("400x300")

        # Rótulo para escolher o tipo de gráfico
        label_tipo = customtkinter.CTkLabel(janela_graficos, text="Tipo de Gráfico:")
        label_tipo.grid(row=0, column=0, padx=10, pady=10)

        # Menu para selecionar o tipo de gráfico
        tipos_graficos = ["Barra", "Linha"]
        self.menu_tipo_grafico = customtkinter.CTkOptionMenu(janela_graficos, values=tipos_graficos)
        self.menu_tipo_grafico.grid(row=0, column=1, padx=10, pady=10)

        # Rótulo para escolher a coluna X
        label_coluna_x = customtkinter.CTkLabel(janela_graficos, text="Coluna X:")
        label_coluna_x.grid(row=1, column=0, padx=10, pady=10)

        # Menu para selecionar a coluna X
        colunas = dataframe.columns.tolist()
        self.menu_coluna_x = customtkinter.CTkOptionMenu(janela_graficos, values=colunas)
        self.menu_coluna_x.grid(row=1, column=1, padx=10, pady=10)

        # Rótulo para escolher a coluna Y
        label_coluna_y = customtkinter.CTkLabel(janela_graficos, text="Coluna Y:")
        label_coluna_y.grid(row=2, column=0, padx=10, pady=10)

        # Menu para selecionar a coluna Y
        self.menu_coluna_y = customtkinter.CTkOptionMenu(janela_graficos, values=colunas)
        self.menu_coluna_y.grid(row=2, column=1, padx=10, pady=10)

        # Botão para gerar o gráfico
        botao_gerar_grafico = customtkinter.CTkButton(janela_graficos, text="Gerar Gráfico", command=self.gerar_grafico_selecionado)
        botao_gerar_grafico.grid(row=3, column=0, columnspan=2, padx=10, pady=10)

    def gerar_grafico_selecionado(self):
        # Usa o último DataFrame carregado da lista `dataframes_carregados`
        if not self.dataframes_carregados:
            tkinter.messagebox.showerror("Erro", "Nenhum DataFrame foi carregado.")
            return

        dataframe = self.dataframes_carregados[-1]  # Usa o último DataFrame carregado

        # Obtém as opções selecionadas
        tipo_grafico = self.menu_tipo_grafico.get()
        coluna_x = self.menu_coluna_x.get()
        coluna_y = self.menu_coluna_y.get()

        # Gera o gráfico conforme o tipo selecionado
        if tipo_grafico == "Barra":
            dataframe.plot(kind="bar", x=coluna_x, y=coluna_y)
        elif tipo_grafico == "Linha":
            dataframe.plot(kind="line", x=coluna_x, y=coluna_y)

        plt.show()


    def gerar_relatorios(self):
        try:
            # Verifica se o DataFrame foi carregado
            if self.dataframes_carregados:
                df = self.dataframes_carregados[-1]  # Usa o último DataFrame carregado

                # Criar uma janela de relatório
                report_window = tk.Toplevel(self)
                report_window.title("Relatório de Dados")
                report_window.geometry("800x600")

                # Adiciona uma label com o nome do arquivo
                file_name_label = tk.Label(report_window, text=f"Relatório para o arquivo: {self.selected_file}", font=("Arial", 14, "bold"))
                file_name_label.pack(pady=10)

                # Exibe as primeiras linhas do DataFrame
                preview_label = tk.Label(report_window, text="Visualização das 5 primeiras linhas:", font=("Arial", 12))
                preview_label.pack(pady=5)
                preview_text = tk.Text(report_window, height=10, width=100)
                preview_text.insert(tk.END, df.head().to_string())
                preview_text.pack(pady=5)

                # Exibe as estatísticas descritivas do DataFrame
                desc_label = tk.Label(report_window, text="Estatísticas Descritivas:", font=("Arial", 12))
                desc_label.pack(pady=5)
                desc_text = tk.Text(report_window, height=10, width=100)
                desc_text.insert(tk.END, df.describe().to_string())
                desc_text.pack(pady=5)

                # Botão para salvar o relatório em um arquivo
                def save_report():
                    save_path = filedialog.asksaveasfilename(defaultextension=".txt", filetypes=[("Arquivo de Texto", "*.txt"), ("Todos os Arquivos", "*.*")])
                    if save_path:
                        with open(save_path, 'w') as f:
                            f.write(f"Relatório para o arquivo: {self.selected_file}\n\n")
                            f.write("Visualização das 5 primeiras linhas:\n")
                            f.write(df.head().to_string())
                            f.write("\n\nEstatísticas Descritivas:\n")
                            f.write(df.describe().to_string())
                        tkinter.messagebox.showinfo("Sucesso", "Relatório salvo com sucesso!")

                # Botão para salvar o relatório
                save_button = customtkinter.CTkButton(report_window, text="Salvar Relatório", command=save_report)
                save_button.pack(pady=10)

            else:
                tkinter.messagebox.showwarning("Aviso", "Nenhum DataFrame foi carregado ainda.")
        
        except Exception as e:
            tkinter.messagebox.showerror("Erro", f"Ocorreu um erro ao gerar o relatório: {str(e)}")


if __name__ == "__main__":
    app = AppSaudados()
    app.mainloop()
